In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import os
import datetime

In [2]:
# File paths for the uploaded Excel files
file_paths = {
    "Genie Dataset 1": "/Users/yongtengchai/Downloads/Genie/Genie Dataset 1.xlsx",
    "Genie Dataset 3": "/Users/yongtengchai/Downloads/Genie/Genie Dataset 3.xlsx",
    "Genie Dataset 4": "/Users/yongtengchai/Downloads/Genie/Genie Dataset 4.xlsx"
}

In [3]:
# Read and preview all datasets
datasets = {}
for name, path in file_paths.items():
    datasets[name] = pd.ExcelFile(path)  # Load the Excel file

# Show the sheet names for each dataset
sheet_info = {name: excel.sheet_names for name, excel in datasets.items()}
sheet_info

{'Genie Dataset 1': ['Sheet'],
 'Genie Dataset 3': ['Sheet'],
 'Genie Dataset 4': ['Sheet']}

In [4]:
# Load the data from the first sheet of each dataset
data_previews = {name: excel.parse(excel.sheet_names[0]).head() for name, excel in datasets.items()}
data_previews

{'Genie Dataset 1':   Date Settled   Agrt No  Reg No.  Agrt Date  Block No  Loan Amt  \
 0   2024-05-06  A0000001  WRJ**** 2022-01-27       NaN     18000   
 1   2023-04-05  A0000002  WTU**** 2022-02-08       NaN     24000   
 2   2024-05-17  A0000003  WXT**** 2022-02-09       NaN      8400   
 3   2024-03-14  A0000004  WVJ**** 2022-02-09       NaN     16200   
 4   2024-03-13  A0000005  WUF**** 2022-02-17       NaN      9840   
 
    Term Charges  Total Payable  Principle Amt  Rebate  ...  O/D Int  Postage  \
 0          9000          27000          11640    2600  ...        8      0.0   
 1          6840          30840          17960    2600  ...        8      0.0   
 2          3360          11760           4100     970  ...        8      0.0   
 3          7700          23900           7160    2500  ...        8      0.0   
 4          4670          14510           3540    1610  ...        8      0.0   
 
    Misc  Repo Exp  Storage  Legal Exp  LOD  JPJ Exp  Dealer ID     Loan Type

In [5]:
# Checking for missing values in each dataset
missing_values_summary = {
    name: df.isnull().sum() for name, df in data_previews.items()
}
missing_values_summary

{'Genie Dataset 1': Date Settled     0
 Agrt No          0
 Reg No.          0
 Agrt Date        0
 Block No         5
 Loan Amt         0
 Term Charges     0
 Total Payable    0
 Principle Amt    0
 Rebate           0
 GL Bal           0
 Last Pd Amt      0
 Tot Instal Pd    0
 O/D Int          0
 Postage          0
 Misc             0
 Repo Exp         0
 Storage          0
 Legal Exp        0
 LOD              0
 JPJ Exp          0
 Dealer ID        0
 Loan Type        0
 dtype: int64,
 'Genie Dataset 3': Branch                0
 Agrt No.              0
 Hirer IC              0
 Age                   0
 State                 0
 Occupation            5
 Marital               0
 Race                  0
 Gender                0
 Total Income          0
 Dealer ID             0
 Reg No.               0
 Car Made              0
 Yr Made               0
 1st Appr Date         5
 Appr Date             0
 Process Officer       0
 Marketing Officer     0
 1st Appr By           5
 Appr By    

In [7]:
# Create a connection to the SQL database
DATABASE_TYPE = 'sqlite'
DB_NAME = 'genie_data.db'
db_url = f"{DATABASE_TYPE}:///{DB_NAME}"
engine = create_engine(db_url, echo=True)

In [8]:
# Read all sheets from all files and insert them into SQL tables
def load_sheets_to_sql(file_path):
    excel_file = pd.ExcelFile(file_path)
    file_name = os.path.basename(file_path).split('.')[0].replace(" ", "_")
    print(f"Processing file: {file_name}")
    for sheet_name in excel_file.sheet_names:
        print(f"Found sheet: {sheet_name}")
        df = excel_file.parse(sheet_name)
        
        # Data Processing: Remove duplicates
        df.drop_duplicates(inplace=True)
        
        # Handle missing values - filling with appropriate values based on column type
        for column in df.columns:
            if df[column].dtype == 'object':
                df[column].fillna(value="Unknown", inplace=True)
            elif df[column].dtype in ['int64', 'float64']:
                df[column].fillna(value=df[column].mean(), inplace=True)
            else:
                df[column].fillna(value=0, inplace=True)
        
        # Additional Data Processing: Convert data types if necessary
        for column in df.columns:
            if df[column].dtype == 'object':
                # Convert categorical text to lowercase to standardize
                df[column] = df[column].str.lower()
            elif df[column].dtype == 'float64':
                # Convert floats to integers if no significant precision is lost
                if (df[column] % 1 == 0).all():
                    df[column] = df[column].astype('int64')
        
        # Clean sheet name for SQL table
        table_name = f"{file_name}_{sheet_name}".replace(" ", "_").lower()
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Loaded sheet '{sheet_name}' into table '{table_name}'")


# Update the table in SQL with processed data

def update_sql_table(df, table_name):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Updated table '{table_name}' with cleaned data")

for file_path in file_paths.values():
    load_sheets_to_sql(file_path)

print("All sheets have been processed and loaded into the SQL database.")


Processing file: Genie_Dataset_1
Found sheet: Sheet
2024-11-24 13:14:01,728 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-24 13:14:01,729 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("genie_dataset_1_sheet")
2024-11-24 13:14:01,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-24 13:14:01,731 INFO sqlalchemy.engine.Engine ROLLBACK
2024-11-24 13:14:01,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-24 13:14:01,731 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("genie_dataset_1_sheet")
2024-11-24 13:14:01,732 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-24 13:14:01,733 INFO sqlalchemy.engine.Engine ROLLBACK
2024-11-24 13:14:01,734 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-24 13:14:01,734 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-11-24 13:14:01,735 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-24 13:14:01,736 INFO sqlalchemy.engin